# Purpose and Objectives:

Welcome! In this notebook, we will be covering the basics of autoencoder architectures, in particular the (vanilla) autoencoder and the variatonal autoencoder architectures. Throughout the notebook, there will be a set of concept check questions. You can find the corresponding question in our supplemental conceptual guide.

After completing this notebook, you will:


1.   Have a complete understanding of the general AE and VAE architectures
2.   Be able to implemting AEs and VAEs in Pytorch
3.   Evaluate the models' performance (loss/accuracy), denoising ability, and latent space through visualizations

*Note: It may be a good idea to review the basic principles of PCA as they will be referred to throughout the project*

## The Fundamental Setup of Autoencoder Architectures:

![AE.jpeg](img/AE.jpeg)

<p style="text-align: center;">Fig 1. A diagram of the general (Vanilla) Autoencoder architecture. </p>

As referenced in Hinton et. al., the purpose of autoencoders aligns very much with the motivations of PCA. One can consider the encoder to be a mapping from the high dimensional space of the input to the low-dimensional space of the latent representation. Similarly, the decoder *reconstructs* the low-dimensional represntation back into the original dimensionality. We discuss the purposes of this architecture in the next section.

### Concept Check 1.1.1

## Why Autoencoder Architectures? (Note to self: add images and figures later)

Autoencoder-style models have many different uses. We list several of the most notable ones here:

1.   **Dimensionality Reduction:** As mentioned in Hinton et. al., the *main* motivation for the AE architecutere is to turn "high-dimensional data" into "low dimensional codes". Adopting from the spirit of PCA, the encoder of an autoencoder can be thought of as a *nonlinear* map of the input data into the compressed space and the decoder an analogous yet opposite map that reconstructs the latent representation into the same dimensionality as the original input. Given sufficient data and computation, AEs can outperform PCA because *AEs are not limited to learning a linear representation of the data*. Datasets that feature high-dimensionality (e.g. biological/genetic data) can benefit from first applying AE dimensionality reduction before applying a later downstream task to the data. In this way, AE architectures can serve as a preprocessing step for other tasks. Note: a more general term for this in research is called **representation learning**.
2.   **Denoising:** Once again, similar to PCA, autoencoders can be utilized as preprocessing method for denoising data. Suppose we have an input vector $\vec{x}_i$ and we add some noise $\vec{\epsilon}$ to it. Then our minimization problem for the autoencoder becomes $\arg\min_{f, g} \sum_{i = 0}^{n}||\vec{x}_i - g(f(\vec{x}_i + \vec{\epsilon}))||^2$. From this objective, we see that an autoencoder used for denoising is motivated to ignore the noise.
3.   **Data Generation:** After properly training an autoencoder architecture, one can ignore the encoder half of the model and sample from the latent representation (or code) distribution. This sampled point can be passed through the decoder to hopefully generate new believable synthetic data.
4.   **Anomaly Detection:** Typically, the problem of anomaly detection involves classification of high dimensional data points (i.e. data with many features) into normal and anomalous samples (e.g. identifying anomolous credit card transaction). At a glance, the interpretations here can be similar to denoising, if we consider anomalies to be a type of "noise". There are many formulations and methematical tricks that can be used for anomaly detection. One common way is for the autoencoder to first learn a representation for the normal data. When running inference, the hope is that the reconstruction error can serve as a metric for evaluating whether a sample is anomolous or not. Intuitively, we would expect normal samples to be close in the low-dimensional space. On the contrary, anomalous samples would be expected to be far from the distribution of normal samples, yielding higher reconstruction error. Over the years many methods that leverage interpolation in the latent space, architectur improvements, and loss functions that incentivize clustering of normal and anomalous samples have been used. If you are curious to learn more about this field of research, refer to the following links: https://paperswithcode.com/task/anomaly-detection, https://github.com/yzhao062/anomaly-detection-resources

### Concept Check 1.1.2

We will now implement the autoencoder and variational autoencoder architectures to work on the MNIST dataset (chosen for its size and ease of visualization).

## AE With Linear Layers on MNIST:

If the following imports don't succeed, please refer to the README to ensure that you properly handled all dependencies. Note: It may take some time to run the imports the first time.

In [ ]:
import math
import random

import numpy as np
import pandas as pd

import sklearn
from sklearn.manifold import TSNE

import torch
from torch import nn
from torch.distributions import MultivariateNormal, Normal, Independent

import torchvision
import torchvision.datasets as datasets
from torchvision import transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
# Reproducability
def reset_seeds():
    random.seed(0) # Python
    torch.manual_seed(0) # Torch
    np.random.seed(0) # NumPy

if not torch.cuda.is_available():
    print("Recommended to train on GPU if possible, will make training process faster")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

We will start with implementing a simple Vanilla Autoencoder model. We will train it on the MNIST dataset. First, let us set up the MNIST dataloaders.

### Set up and visualize data

In [ ]:
# AE hyperparams
epochs = 5
lr = 1e-3
batch_size = 64
input_dim = 784
latent_dim = 512
noise = False

In [ ]:
# data
train_dataset = datasets.MNIST(root='data/', download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='data/', train=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Visualize the data:

# Grab first batch of images:
images, labels = next(iter(train_dataloader))
    
# Print and show the first 10 samples:

print(f"Labels: {labels[0:10]}")
im = make_grid(images[:10], nrow=10)
plt.figure(figsize=(10,1))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0))) #Remember that default MNIST data is CWH, but matplotlib uses WHC

Now that we have set up our data, let us construct the Vanilla Autoencoder with linear layers. We know that the input to the first layer should be the flattened MNIST data dimension. Following this, the layers of the AE encoder get iteratively smaller until the bottleneck (last layer of our encoder), which dictates the latent dimension size. The decoder is a mirror reflection of the encoder. Refer to [Hinton et. al.](https://www.science.org/doi/10.1126/science.1127647) for an idea on standard architecture if you are stuck. You may also utilize the helper code MLP, but it is not required.

### AE Model & Train

In [ ]:
#Some Optional Helper Code
def mlp(in_dim, out_dim, encode=True):
    # A helper function that compactly creates an MLP
    # where each layer is half of (or double) the size of the previous layer
    # depending on `encode`
    layers = []
    if encode:
        next_dim = 2**math.floor(math.log2(in_dim))
    else:
        next_dim = 2**math.ceil(math.log2(in_dim))
        
    firstLayer = nn.Linear(in_dim, next_dim)
    layers.extend([firstLayer, nn.ReLU()])
    in_dim = next_dim
        
    while in_dim != out_dim:
        if encode:
            next_dim = in_dim // 2
            if next_dim < out_dim:
                next_dim = out_dim
        else:
            next_dim = in_dim * 2
            if next_dim > out_dim:
                next_dim = out_dim
        layers.extend([nn.Linear(in_dim, next_dim), nn.ReLU()])
        in_dim = next_dim
        
    return nn.Sequential(*layers)

In [ ]:
class AE(nn.Module):
    def __init__(self, in_dim=784, latent_dim=512):
        super().__init__()
        

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)
    
    def train_step(self, optimizer, x_in, x_star):
        z = self.encode(x_in)
        x_hat = self.decode(z)

        loss = self.loss(x_star, x_hat)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss
    
    def test_step(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        
        loss = self.loss(x, x_hat)
        return loss
    
    @staticmethod
    def loss(x, x_hat):
        loss = None

        # TODO: Mean-Squared Error Reconstruction Loss
        raise NotImplementedError()
        
        return loss



The training loop is set up for you. Feel free to tinker with the hyperparameters + model architecture at this point if you would like. On the default settings (with CUDA), this cell should take just over a minute to run.

In [ ]:
#Training Loop for Autoencoder

# model and optimizer
reset_seeds()
ae = AE(784, 512)
ae.to(device)
optimizer = torch.optim.Adam(ae.parameters(), lr=lr)

# train
ae_train_losses = []
ae_test_losses = []
step = 0
report_every = 500
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    # train loss:
    for x, y in tqdm(train_dataloader):
        x_no_noise = x.reshape(x.shape[0], -1)  # flatten
        if noise:
            x_in = add_noise(x_no_noise, noise_weight=0.5)
        else:
            x_in = x_no_noise
        
        x_in = x_in.to(device)
        x_no_noise = x_no_noise.to(device)
        loss = ae.train_step(optimizer, x_in=x_in, x_star=x_no_noise)
        ae_train_losses.append(loss.item()) # loss every iteration
        step += 1
        if step % report_every == 0:
            tqdm.write(f"Training loss: {loss}")
    # ae_train_losses.append(loss.detach().numpy()) # loss after every epoch
    # test loss
    for b, (X_test, y_test) in enumerate(test_dataloader):
        X_test = X_test.reshape(X_test.shape[0], -1)  # flatten
        X_test = X_test.to(device)
        loss = ae.test_step(X_test)
        ae_test_losses.append(loss.item()) # loss every iteration
    # ae_test_losses.append(loss.detach().numpy()) # loss after every epoch

### Thinking Critically About the Architecture:

Alright, we have now trained our linear-layer (Vanilla) Autoencoder! Before directly moving on to Variational Autoencoders, let us see if we can't critically analyze the benefits and shortcomings of the Autoencoder.

Let us think a little more closely about the latent representation of the Autoencoder. We note that besides optimizing the reconstruction loss, there are no further restrictions on the latent code. Because of this, we can't really ensure that the latent space representation being learned is anything semantically meaningful or even practically useful (besides being good at reconstruction of course!).

Due to this lack of an informative latent space, (Vanilla) Autoencoders have more trouble with **denoising** and **content generation** tasks. One way to fix this is to regularize the latent code using some prior. 

![VAE_diagram.jpeg](img/VAE_diagram.jpeg)

<p style="text-align: center;">Fig 2. A diagram of the VAE architecture. </p>

Finally, to enforce this regularization, our loss objective thus becomes:

$$\|X - g(f(X))\|^2 + \lambda\mathrm{KL}[\mathcal{N}(\mu,\,\sigma^{2}), \mathcal{N}(0,\,I)]$$

where $\mathrm{KL}(\cdot)$ represents the KL-divergence. Without loss of generality, we measure the KL divergence between our latent distribution and the standard normal distribution.

### Concept Check 2.1

## VAE With Linear Layers on MNIST:

### VAE Model & Train

In [ ]:
# VAE hyperparams
epochs = 5
lr = 1e-3
batch_size = 64
noise = False
input_dim = 784
latent_dim=512
regularization_weight = .0001

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim=1*28*28, latent_dim=512):
        super().__init__()
        self.z_mean = mlp(input_dim, latent_dim, encode=True)
        self.z_log_std = mlp(input_dim, latent_dim, encode=True)
        self.decoder = mlp(latent_dim, input_dim, encode=False)
    
    def _encode(self, x):
        # Output: latent_features is output of sampling q(z|x), log_prob 
        # Hint: Check out torch.distributions, in particular Independent and Normal
        # for calculating the log likelihood of your sampled z under q(z|x)
        latent_features = None
        log_prob = None

        raise NotImplementedError()
        return latent_features, log_prob 
        
    def encode(self, x):
        z, _ = self._encode(x)
        return z

    def decode(self, z):
        return self.decoder(z)
    
    def train_step(self, optimizer, x_in, x_star):
        z, log_prob = self._encode(x_in)
        x_hat = self.decode(z)

        loss = self.loss(x_star, x_hat, z, log_prob)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        return loss
    
    def test_step(self, x):
        z, log_prob = self._encode(x)
        x_hat = self.decode(z)
        
        loss = self.loss(x, x_hat, z, log_prob)
        return loss

    @staticmethod
    def loss(x, x_hat, z, log_prob, kl_weight=regularization_weight):
        # Output: loss given true data x, reconstructed data x_hat, 
        # sampled latent variables z, log likelihood of z under q(z|x), 
        # and the weight of the KL divergence term
        # Hint: check out torch.distributions, again specifically the MultivariateNormal
        # class to calculate the log likelihood of latent variables under a multivariate standard normal
        loss = None
        raise NotImplementedError()
        return loss

Again, feel free to tinker with the architecture or the hyperparameters if you would like. With the default settings, you should expect the following cell to take ~90 seconds to run

In [ ]:
# Training Loop for VAE

# model and optimizer
reset_seeds()
vae = VAE(input_dim, latent_dim)
vae.to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=lr)

# train
vae_train_losses = []
vae_test_losses = []
step = 0
report_every = 500
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    # train loss:
    for x, y in tqdm(train_dataloader):
        x_no_noise = x.reshape(x.shape[0], -1)  # flatten
        if noise:
            x_in = add_noise(x_no_noise, noise_weight=0.5)
        else:
            x_in = x_no_noise
        x_in, x_no_noise = x_in.to(device), x_no_noise.to(device)
        loss = vae.train_step(optimizer, x_in=x_in, x_star=x_no_noise)
        vae_train_losses.append(loss.item()) #loss every iteration
        step += 1
        if step % report_every == 0:
            tqdm.write(f"Training loss: {loss}")
    # vae_train_losses.append(loss.detach().numpy()) #loss after every epoch
    # test loss
    for b, (X_test, y_test) in enumerate(test_dataloader):
        X_test = X_test.reshape(X_test.shape[0], -1)  # flatten
        X_test = X_test.to(device)
        loss = vae.test_step(X_test)
        vae_test_losses.append(loss.item()) #loss every iteration
    # vae_test_losses.append(loss.detach().numpy()) #loss after every epoch

### Concept Check 2.2-2.3 (while you wait for the model to train)

## Ablations and Visualizations:

If you would like to understand more how these visualizations and ablations were implemented, feel free to peruse the code. However, for the most part take a close, critical look at the results. Some of these are really interesting and informative!

### 1. Loss Visualization:

In [ ]:
with torch.no_grad():
    fig, ax = plt.subplots(2, figsize=(8,10))
    ax[0].plot(ae_train_losses, label="AE")
    ax[0].plot(vae_train_losses, label="VAE")
    ax[0].set_title("Train Losses")
    ax[0].set_ylabel("Train Loss")
    ax[0].set_xlabel("Iteration")
    ax[0].legend()
    
    ax[1].plot(ae_test_losses, label="AE")
    ax[1].plot(vae_test_losses, label="VAE")
    ax[1].set_title("Test Losses")
    ax[1].set_ylabel("Test Loss")
    ax[1].set_xlabel("Iteration")
    ax[1].legend()

#Note: if you want per epoch (or avg. epoch) losses, you will have to change the above code somewhat

### 2. Visualize Reconstructed Samples

In [ ]:
# Visualize the samples in bulk:
with torch.no_grad():
    # Grab first batch of images:
    images, labels = next(iter(test_dataloader))
    flattened = images.reshape((batch_size, -1))
    flattened = flattened.to(device)
    recon_ae = ae.decode(ae.encode(flattened)).reshape((batch_size, 1, 28, 28))
    recon_vae = vae.decode(vae.encode(flattened)).reshape((batch_size, 1, 28, 28))
    
    # Print and show the first 10 samples:
    print(f"Labels: {labels[0:10]}")
    im = make_grid(images[:10], nrow=10)
    ae_im = make_grid(recon_ae[:10], nrow=10)
    vae_im = make_grid(recon_vae[:10], nrow=10)
    fig, ax = plt.subplots(3, figsize=(45,4.5))
    fig.tight_layout(pad=1.5)
    ax[0].imshow(np.transpose(im.cpu().numpy(), (1, 2, 0))) #Remember that default MNIST data is CWH, but matplotlib uses WHC
    ax[0].set_title("Original:")

    ax[1].imshow(np.transpose(ae_im.cpu().numpy(), (1, 2, 0)))
    ax[1].set_title("AE Reconstruction:")
    
    ax[2].imshow(np.transpose(vae_im.cpu().numpy(), (1, 2, 0)))
    ax[2].set_title("VAE Reconstruction:")

### Concept Check 2.4.1

### 3. Visualizing the Latent Space:

In [ ]:
# analysis
with torch.no_grad():
    num_samples = 512

    viz_dataloader = DataLoader(test_dataset, batch_size=num_samples, shuffle=True)
    data, labels = next(iter(viz_dataloader))
    tsne = TSNE(n_components=2, perplexity=30.0, random_state=0)
    flattened_data = data.reshape((num_samples, -1))
    flattened_data = flattened_data.to(device)
    recon_ae = ae.encode(flattened_data)
    recon_vae = vae.encode(flattened_data)
    embedded_ae = tsne.fit_transform(recon_ae.cpu())
    embedded_vae = tsne.fit_transform(recon_vae.cpu())
    two_components_ae = np.hstack((embedded_ae, labels.numpy().reshape(-1, 1)))
    two_components_ae = pd.DataFrame(data=two_components_ae)
    two_components_ae.columns = ["c1", "c2", "label"]

    label_groups_ae = two_components_ae.groupby("label")
    
    two_components_vae = np.hstack((embedded_vae, labels.numpy().reshape(-1, 1)))
    two_components_vae = pd.DataFrame(data=two_components_vae)
    two_components_vae.columns = ["c1", "c2", "label"]

    label_groups_vae = two_components_vae.groupby("label")
    
    fig, ax = plt.subplots(2, figsize=(8,16))
    fig.tight_layout(pad=1.75)
    #plt.figure(figsize=(10, 10))
    for label, group in label_groups_ae:
        ax[0].scatter(group["c1"], group["c2"], marker="o", label=label)

    ax[0].legend(labels=range(10), fontsize=18)
    ax[0].set_title("AE t-SNE Clustering", fontdict={"fontsize": 24})
    #ax[0].show()
    
    for label, group in label_groups_vae:
        ax[1].scatter(group["c1"], group["c2"], marker="o", label=label)

    ax[1].legend(labels=range(10), fontsize=18)
    ax[1].set_title("VAE t-SNE Clustering", fontdict={"fontsize": 24})
    #ax[1].show()

In [ ]:
# analysis
with torch.no_grad():
    num_samples = 512

    viz_dataloader = DataLoader(test_dataset, batch_size=num_samples, shuffle=True)
    data, labels = next(iter(viz_dataloader))

    tsne = TSNE(n_components=3, perplexity=30.0, random_state=0)
    flattened_data = data.reshape((num_samples, -1))
    flattened_data = flattened_data.to(device)
    recon_ae = ae.encode(flattened_data)
    recon_vae = vae.encode(flattened_data)
    embedded_ae = tsne.fit_transform(recon_ae.cpu())
    embedded_vae = tsne.fit_transform(recon_vae.cpu())
    two_components_ae = np.hstack((embedded_ae, labels.numpy().reshape(-1, 1)))
    two_components_ae = pd.DataFrame(data=two_components_ae)
    two_components_ae.columns = ["c1", "c2", "c3", "label"]

    label_groups_ae = two_components_ae.groupby("label")
    
    two_components_vae = np.hstack((embedded_vae, labels.numpy().reshape(-1, 1)))
    two_components_vae = pd.DataFrame(data=two_components_vae)
    two_components_vae.columns = ["c1", "c2","c3", "label"]

    label_groups_vae = two_components_vae.groupby("label")
    
    fig = plt.figure(figsize=plt.figaspect(0.5))
    ax0 = fig.add_subplot(1, 2, 1, projection='3d')
    fig.tight_layout(pad=1.75)
    #plt.figure(figsize=(10, 10))
    for label, group in label_groups_ae:
        ax0.scatter(group["c1"], group["c2"], group["c3"], marker="o", label=label)

    ax0.legend(labels=range(10), fontsize=14)
    ax0.set_title("AE t-SNE Clustering", fontdict={"fontsize": 18})
    #ax[0].show()
    
    ax1 = fig.add_subplot(1, 2, 2, projection='3d')
    for label, group in label_groups_vae:
        ax1.scatter(group["c1"], group["c2"], group["c3"], marker="o", label=label)

    ax1.legend(labels=range(10), fontsize=14)
    ax1.set_title("VAE t-SNE Clustering", fontdict={"fontsize": 18})
    #ax[1].show()

### Concept Check 2.4.2

### 4. AE/VAE Denoising:

Now, let us see the denoising capabilities of these architectures. Go back to the function add noise in order to add some Gaussian noise to the training data, $X$. You can adjust how reasonable the noise is by adding a coefficient in front of the noise term, which can act as a hyperparameter. Then, apply this noise to the sample when training.

*(Hint: torch.randn may be helpful here. Also, consider what range of values may be outputted when you add noise. Is this consistent with the data type of MNIST? How might you account for this.)*

In [ ]:
def add_noise(tensor, mean=0., std=1., noise_weight=0.5):
    # TODO
    raise NotImplementedError()

In [ ]:
noise = True
epochs = 20


In [ ]:
#@title The training loops for AE and VAE are replicated below for your convenience
#Training Loop for Autoencoder

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

#Training Loop for Autoencoder

# model and optimizer
reset_seeds()
ae = AE(784, 512)
ae.to(device)
optimizer = torch.optim.Adam(ae.parameters(), lr=lr)

# train
ae_train_losses = []
ae_test_losses = []
step = 0
report_every = 500
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    # train loss:
    for x, y in tqdm(train_dataloader):
        x_no_noise = x.reshape(x.shape[0], -1)  # flatten
        if noise:
            x_in = add_noise(x_no_noise, noise_weight=0.5)
        else:
            x_in = x_no_noise
        
        x_in = x_in.to(device)
        x_no_noise = x_no_noise.to(device)
        loss = ae.train_step(optimizer, x_in=x_in, x_star=x_no_noise)
        ae_train_losses.append(loss.item()) # loss every iteration
        step += 1
        if step % report_every == 0:
            tqdm.write(f"Training loss: {loss}")
    # ae_train_losses.append(loss.detach().numpy()) # loss after every epoch
    # test loss
    for b, (X_test, y_test) in enumerate(test_dataloader):
        X_test = X_test.reshape(X_test.shape[0], -1)  # flatten
        X_test = X_test.to(device)
        loss = ae.test_step(X_test)
        ae_test_losses.append(loss.item()) # loss every iteration
    # ae_test_losses.append(loss.detach().numpy()) # loss after every epoch

# Training Loop for VAE

# model and optimizer
reset_seeds()
vae = VAE(input_dim, latent_dim)
vae.to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=lr)

# train
vae_train_losses = []
vae_test_losses = []
step = 0
report_every = 500
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    # train loss:
    for x, y in tqdm(train_dataloader):
        x_no_noise = x.reshape(x.shape[0], -1)  # flatten
        if noise:
            x_in = add_noise(x_no_noise, noise_weight=0.5)
        else:
            x_in = x_no_noise
        x_in, x_no_noise = x_in.to(device), x_no_noise.to(device)
        loss = vae.train_step(optimizer, x_in=x_in, x_star=x_no_noise)
        vae_train_losses.append(loss.item()) #loss every iteration
        step += 1
        if step % report_every == 0:
            tqdm.write(f"Training loss: {loss}")
    # vae_train_losses.append(loss.detach().numpy()) #loss after every epoch
    # test loss
    for b, (X_test, y_test) in enumerate(test_dataloader):
        X_test = X_test.reshape(X_test.shape[0], -1)  # flatten
        X_test = X_test.to(device)
        loss = vae.test_step(X_test)
        vae_test_losses.append(loss.item()) #loss every iteration
    # vae_test_losses.append(loss.detach().numpy()) #loss after every epoch

In [ ]:
with torch.no_grad():
    fig, ax = plt.subplots(2, figsize=(8,10))
    ax[0].plot(ae_train_losses, label="AE")
    ax[0].plot(vae_train_losses, label="VAE")
    ax[0].set_title("Train Losses")
    ax[0].set_ylabel("Train Loss")
    ax[0].set_xlabel("Iteration")
    ax[0].legend()

    ax[1].plot(ae_test_losses, label="AE")
    ax[1].plot(vae_test_losses, label="VAE")
    ax[1].set_title("Test Losses")
    ax[1].set_ylabel("Test Loss")
    ax[1].set_xlabel("Iteration")
    ax[1].legend()

#Note: if you want per epoch (or avg. epoch) losses, you will have to change the above code somewhat

In [ ]:
# Visualize the samples in bulk:

with torch.no_grad():
    # Grab first batch of images:
    images, labels = next(iter(test_dataloader))


    # Add Noise to images:
    noise_images = add_noise(images, noise_weight=0.65)
    flattened_noise_images = noise_images.reshape((batch_size, -1))
    flattened_noise_images = flattened_noise_images.to(device)
    recon_ae = ae.decode(ae.encode(flattened_noise_images)).reshape((batch_size, 1, 28, 28))
    recon_vae = vae.decode(vae.encode(flattened_noise_images)).reshape((batch_size, 1, 28, 28))
    
    # Print and show the first 10 samples:
    print(f"Labels: {labels[0:10]}")
    im = make_grid(images[:10], nrow=10)
    noise_im = make_grid(noise_images[:10], nrow=10)
    ae_im = make_grid(recon_ae[:10], nrow=10)
    vae_im = make_grid(recon_vae[:10], nrow=10)
    fig, ax = plt.subplots(4, figsize=(60,6))
    fig.tight_layout(pad=1.5)
    ax[0].imshow(np.transpose(im.cpu().numpy(), (1, 2, 0))) #Remember that default MNIST data is CWH, but matplotlib uses WHC
    ax[0].set_title("Original:")
    
    ax[1].imshow(np.transpose(noise_im.cpu().numpy(), (1, 2, 0)))
    ax[1].set_title("With added noise:")

    ax[2].imshow(np.transpose(ae_im.cpu().numpy(), (1, 2, 0)))
    ax[2].set_title("AE Reconstruction:")
    
    ax[3].imshow(np.transpose(vae_im.cpu().numpy(), (1, 2, 0)))
    ax[3].set_title("VAE Reconstruction:")

### Concept Check 2.3.3

In [ ]:
# analysis
with torch.no_grad():
    num_samples = 512

    viz_dataloader = DataLoader(test_dataset, batch_size=num_samples, shuffle=True)
    data, labels = next(iter(viz_dataloader))
    flattened_data = data.reshape((num_samples, -1))
    flattened_data = flattened_data.to(device)

    tsne = TSNE(n_components=2, perplexity=30.0, random_state=0)
    recon_ae = ae.encode(flattened_data)
    recon_vae = vae.encode(flattened_data)
    embedded_ae = tsne.fit_transform(recon_ae.cpu())
    embedded_vae = tsne.fit_transform(recon_vae.cpu())
    two_components_ae = np.hstack((embedded_ae, labels.numpy().reshape(-1, 1)))
    two_components_ae = pd.DataFrame(data=two_components_ae)
    two_components_ae.columns = ["c1", "c2", "label"]

    label_groups_ae = two_components_ae.groupby("label")
    
    two_components_vae = np.hstack((embedded_vae, labels.numpy().reshape(-1, 1)))
    two_components_vae = pd.DataFrame(data=two_components_vae)
    two_components_vae.columns = ["c1", "c2", "label"]

    label_groups_vae = two_components_vae.groupby("label")
    
    fig, ax = plt.subplots(2, figsize=(8,16))
    fig.tight_layout(pad=1.75)
    #plt.figure(figsize=(10, 10))
    for label, group in label_groups_ae:
        ax[0].scatter(group["c1"], group["c2"], marker="o", label=label)

    ax[0].legend(labels=range(10), fontsize=18)
    ax[0].set_title("AE t-SNE Clustering", fontdict={"fontsize": 24})
    #ax[0].show()
    
    for label, group in label_groups_vae:
        ax[1].scatter(group["c1"], group["c2"], marker="o", label=label)

    ax[1].legend(labels=range(10), fontsize=18)
    ax[1].set_title("VAE t-SNE Clustering", fontdict={"fontsize": 24})
    #ax[1].show()

In [ ]:
# analysis
with torch.no_grad():
    num_samples = 512

    viz_dataloader = DataLoader(test_dataset, batch_size=num_samples, shuffle=True)
    data, labels = next(iter(viz_dataloader))

    flattened_data = data.reshape((num_samples, -1)).to(device)
    tsne = TSNE(n_components=3, perplexity=30.0, random_state=0)
    recon_ae = ae.encode(flattened_data)
    recon_vae = vae.encode(flattened_data)
    embedded_ae = tsne.fit_transform(recon_ae.cpu())
    embedded_vae = tsne.fit_transform(recon_vae.cpu())
    two_components_ae = np.hstack((embedded_ae, labels.numpy().reshape(-1, 1)))
    two_components_ae = pd.DataFrame(data=two_components_ae)
    two_components_ae.columns = ["c1", "c2", "c3", "label"]

    label_groups_ae = two_components_ae.groupby("label")
    
    two_components_vae = np.hstack((embedded_vae, labels.numpy().reshape(-1, 1)))
    two_components_vae = pd.DataFrame(data=two_components_vae)
    two_components_vae.columns = ["c1", "c2","c3", "label"]

    label_groups_vae = two_components_vae.groupby("label")
    
    fig = plt.figure(figsize=plt.figaspect(0.5))
    ax0 = fig.add_subplot(1, 2, 1, projection='3d')
    fig.tight_layout(pad=1.75)
    #plt.figure(figsize=(10, 10))
    for label, group in label_groups_ae:
        ax0.scatter(group["c1"], group["c2"], group["c3"], marker="o", label=label)

    ax0.legend(labels=range(10), fontsize=14)
    ax0.set_title("AE t-SNE Clustering", fontdict={"fontsize": 18})
    #ax[0].show()
    
    ax1 = fig.add_subplot(1, 2, 2, projection='3d')
    for label, group in label_groups_vae:
        ax1.scatter(group["c1"], group["c2"], group["c3"], marker="o", label=label)

    ax1.legend(labels=range(10), fontsize=14)
    ax1.set_title("VAE t-SNE Clustering", fontdict={"fontsize": 18})
    #ax[1].show()

### 5. Generating New Samples

#### 5a. AE:

In [ ]:
ae.eval()

with torch.no_grad():
    # calculate mean and std of latent code, generated takining in test images as inputs 
    images, labels = next(iter(test_dataloader))
  
    flattened_images = images.reshape((batch_size, -1)).to(device)
    latent = vae.encode(flattened_images)
    latent = latent.cpu()
    mean = latent.mean(dim=0)
    std = (latent - mean).pow(2).mean(dim=0).sqrt()

    # sample latent vectors from the normal distribution
    latent = torch.randn(batch_size, latent_dim)*std + mean

    # reconstruct images from the random latent vectors
    latent = latent.to(device)
    img_recon = ae.decode(latent)
    img_recon = img_recon.reshape((batch_size, 1, 28, 28)).cpu()

    fig, ax = plt.subplots(figsize=(20, 8.5))
    im = make_grid(img_recon[:20], nrow=10)
    plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))
    
    plt.show()

#### 5b. VAE:

In [ ]:
vae.eval()

with torch.no_grad():
    # calculate mean and std of latent code, generated takining in test images as inputs 
    images, labels = next(iter(test_dataloader))
    images = images.to(device)
    latent = vae.encode(images.reshape((batch_size, -1)))
    latent = latent.cpu()
    mean = latent.mean(dim=0)
    std = (latent - mean).pow(2).mean(dim=0).sqrt()

    # sample latent vectors from the normal distribution
    latent = torch.randn(batch_size, latent_dim)*std + mean

    # reconstruct images from the random latent vectors
    latent = latent.to(device)
    img_recon = vae.decode(latent)
    img_recon = img_recon.reshape((batch_size, 1, 28, 28)).cpu()

    fig, ax = plt.subplots(figsize=(20, 8.5))
    im = make_grid(img_recon[:20], nrow=10)
    plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))
    
    plt.show()

### Concept Check 2.3.4

## References:

Any references utilized in this project can be found in the README of our repo.

In [ ]:
#@title A faster version (no training losses, only denoising, AE)
#Training Loop for Autoencoder

# model and optimizer
reset_seeds()
ae = AE(784, 512)
ae.to(device)
optimizer = torch.optim.Adam(ae.parameters(), lr=lr, capturable=True)

warmup_iterator = iter(train_dataloader)
x, y = next(warmup_iterator)
x_no_noise = x.reshape(x.shape[0], -1)
x_in = add_noise(x_no_noise, noise_weight=0.5)
static_input = torch.zeros_like(x_in, device=device).copy_(x_in)
static_target = torch.zeros_like(x_no_noise, device=device).copy_(x_no_noise)

s = torch.cuda.Stream()
s.wait_stream(torch.cuda.current_stream())
with torch.cuda.stream(s):
    for i in range(3):
        x, _ = next(warmup_iterator)
        x_no_noise = x.reshape(x.shape[0], -1)
        x_in = add_noise(x_no_noise, noise_weight=0.5)
        x_in = x_in.to(device)
        x_no_noise = x_no_noise.to(device)
        loss = ae.train_step(optimizer, x_in=x_in, x_star=x_no_noise)
torch.cuda.current_stream().wait_stream(s)

# capture
g = torch.cuda.CUDAGraph()
# Sets grads to None before capture, so backward() will create
# .grad attributes with allocations from the graph's private pool
optimizer.zero_grad(set_to_none=True)
with torch.cuda.graph(g):
    static_input = add_noise(static_target, noise_weight=0.5)
    loss = ae.train_step(optimizer, x_in=static_input, x_star=static_target)

# train
step = 0
report_every = 500
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    # train loss:
    for x, y in tqdm(train_dataloader):
        x_no_noise = x.reshape(x.shape[0], -1)  # flatten
        if x.shape != static_input.shape:
          continue
        static_target.copy_(x_no_noise)

        g.replay()
        
        step += 1
        if step % report_every == 0:
            tqdm.write(f"Training loss: {loss}")
    # ae_train_losses.append(loss.detach().numpy()) # loss after every epoch
    # test loss
    for b, (X_test, y_test) in enumerate(test_dataloader):
        X_test = X_test.reshape(X_test.shape[0], -1)  # flatten
        X_test = X_test.to(device)
        loss = ae.test_step(X_test)
        ae_test_losses.append(loss.item()) # loss every iteration
    # ae_test_losses.append(loss.detach().numpy()) # loss after every epoch